In [12]:
from pkg_resources import non_empty_lines
%pip install -q peft datasets evaluate
%pip install -q transformers[torch]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate
from peft import get_peft_model, LoraConfig, TaskType
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(torch.cuda.is_available())
print(torch.version.cuda)


False
None


In [3]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
dataset = load_dataset("glue", "mrpc")
model_name_or_path = "roberta-base"
task = "mrpc"
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [4]:
metric = evaluate.load("glue", task)

In [26]:
dataset["validation"][1]

{'sentence1': 'Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .',
 'sentence2': 'His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .',
 'label': 0,
 'idx': 18}

In [29]:
metric

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value('int64'), 'references': Value('int64')}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results

In [30]:
references = [0, 1]
predictions = [1, 1]
results = metric.compute(predictions=predictions, references=references)
print(results)

{'accuracy': 0.5, 'f1': 0.6666666666666666}


In [ ]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side = "right")

if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        max_length = None
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["sentence1", "sentence2"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [33]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [39]:
tokenized_datasets["train"][1]

{'labels': 0,
 'idx': 1,
 'input_ids': [0,
  975,
  26802,
  1588,
  102,
  2164,
  13976,
  1758,
  128,
  29,
  137,
  2183,
  5,
  3206,
  7,
  11881,
  10564,
  11,
  6708,
  13,
  68,
  132,
  4,
  245,
  325,
  479,
  2,
  2,
  975,
  26802,
  1588,
  102,
  2162,
  13976,
  1758,
  128,
  29,
  11,
  7969,
  13,
  68,
  231,
  6478,
  153,
  8,
  1088,
  24,
  7,
  11881,
  10564,
  13,
  68,
  112,
  4,
  398,
  325,
  11,
  6708,
  479,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [47]:
example_input_ids = tokenized_datasets['train'][0]['input_ids']
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(example_input_ids))



'<s>Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .</s></s>Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .</s>'

In [48]:
#from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")


In [52]:
#from transformers import AutoModelForSequenceClassification, TrainingArguments, trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True)
print(sum(p.numel() for p in model.parameters()))
training_args = TrainingArguments(
    output_dir="./output",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy = "epoch",
    save_strategy="epoch",
    logging_steps=100,
    load_best_model_at_end=True,
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


124647170


In [56]:
from transformers import trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions = predictions, references = labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],  # or "test" if you prefer
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics= compute_metrics,
)

trainer.train()
for log in trainer.state.log_history:
    if "epoch" in log:
        print(f"Epoch: {log['epoch']}, Train Loss: {log.get('loss')}, Val Loss: {log.get('eval_loss')}, Accuracy: {log.get('eval_accuracy')}, F1: {log.get('eval_f1')}")



C:\Users\Balle\AppData\Local\Temp\ipykernel_19480\2670372403.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [23]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6831488609313965, 'eval_model_preparation_time': 0.0017, 'eval_accuracy': 0.6838235294117647, 'eval_f1': 0.8122270742358079, 'eval_runtime': 11.3143, 'eval_samples_per_second': 36.061, 'eval_steps_per_second': 1.149}
